In [1266]:
from random import random, choice, randint
from collections import namedtuple
from pprint import pprint, pformat
import random
import numpy as np
from copy import copy, deepcopy
import logging
from dataclasses import dataclass


In [1267]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [1268]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self): #The __bool__ method returns True if there are objects remaining in any row, indicating that the game is not over
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


In [1269]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [1270]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## TASK 1

In [1271]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .80
size_game=4


In [1272]:

@dataclass
class PossibleMove():
    nim_move: Nimply
    fitness: int
    


def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]        #list of possibleMove elements
    champion = max(pool, key=lambda possiblemove: possiblemove.fitness)     #one possibleMove element with the best nim_sum
    return champion


def mutate(move: PossibleMove, nim_state: Nim) -> PossibleMove:     #mutates the number of objects to remove from the same unchanged row OR mutates the row 
    offspring = copy(move)

    test =offspring.nim_move.num_objects 

    if(test +1 <= nim_state.rows[offspring.nim_move.row]):
        offspring.nim_move=offspring.nim_move._replace(num_objects=offspring.nim_move.num_objects+1)

    elif(test == nim_state.rows[offspring.nim_move.row] and test>=2): 
        offspring.nim_move=offspring.nim_move._replace(num_objects=offspring.nim_move.num_objects-1)



    #elif ((offspring.nim_move.num_objects +1)<= nim_state.rows[offspring.nim_move.row] and (offspring.nim_move.num_objects +1)<=nim_state._k):
        #offspring.nim_move.num_objects+=1

    offspring.fitness = None
    return offspring




In [1273]:
def fitness(nim_state : Nim , move : Nimply):
    copy_game= deepcopy(nim_state)
    copy_game.nimming(move)
    nim_sum_val=nim_sum(copy_game)
    incr=0
    all_ones=[]
    if (nim_sum_val==1):
        for element in copy_game._rows:
            if element != 0:
                incr+=1
                all_ones.append(element)

        if incr%2 ==1 and sum(1 for element in all_ones if element != 1) ==1:
            return -1

        
    return(nim_sum_val)
    

In [1274]:
def nimsum_evolved_startegy(nim_state:Nim):
    # Create a list of PossibleMove instances, each representing a possible move
    population = [
        PossibleMove(
            fitness=None,
            nim_move=pure_random(nim_state)                
        )
        for _ in range(POPULATION_SIZE)
    ]

    for i in population:
        i.fitness = fitness(nim_state, i.nim_move)  # Use the fitness function on the nim_move

    for generation in range(10):
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            p1 = select_parent(population)
            if random.random() < MUTATION_PROBABILITY:     #based on nim_sum                             
                p1 = mutate(p1,nim_state)
                            
            offspring.append(p1)

        for i in offspring:
            i.fitness = fitness(nim_state, i.nim_move) # Use the fitness function on the nim_move
            
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness)            #I don't think here we should add the parameter "reverse=True"
        population = population[:POPULATION_SIZE]

    population.sort(key=lambda i: i.fitness)
    # Choose the best individual in the population and return its nim_move
    print('best ind')
    print(population[0].fitness)
    best_individual = population[0]
    return best_individual.nim_move

## Oversimplified match

In [1275]:
logging.getLogger().setLevel(logging.INFO)


# Choose the best PossibleMove in the population and play Nim against the pure random strategy
strategy = ( nimsum_evolved_startegy, pure_random)

nim = Nim(size_game)
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 7>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 5 7>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 2 4 7>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <0 2 2 7>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7)
INFO:root:status: <0 2 2 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=2)
INFO:root:status: <0 0 2 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=2)
INFO:root:status: <0 0 0 0>
INFO:root:status: Player 1 won!


best ind
1
best ind
1
best ind
0
best ind
0
